In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
from code_class import GeoVictoria, FolhaPonto
from tqdm import tqdm
import datetime as dt
import codeshot

pd.set_option('display.max_rows', 100)

In [ ]:
dias_semana = [(   'Sunday',       'Domingo'),
               (   'Monday', 'Segunda-feira'),
               (  'Tuesday',   'Terça-feira'),
               ('Wednesday',  'Quarta-feira'),
               ( 'Thursday',  'Quinta-feira'),
               (   'Friday',   'Sexta-feira'),
               ( 'Saturday',        'Sábado')]

df_data = pd.DataFrame(data=[dt.date(year=2022, month=11, day=21) + dt.timedelta(days=x) for x in range(0, 182)], columns=['Data'])

In [ ]:
lista_tp_permis = []

for permission in GeoVictoria.GetPermissionList():
    lista_tp_permis.append((permission['ID_TIPO_PERMISO'], permission['DESCRIPCION_TIPO_PERMISO']))

In [ ]:
df_funcionários = pd.DataFrame(GeoVictoria.GetUserList())

df_funcionários['Setor'] = df_funcionários['GroupDescription'].apply(lambda x:  '' if 'LOGIN' in x.upper() else
                                                                                'Garantia' if 'GARAN' in x.upper() else
                                                                                'Logistica' if 'LOGIS' in x.upper() else
                                                                                'Logistica' if 'LOGÍS' in x.upper() else
                                                                                'Vendas' if 'VENDA' in x.upper() else
                                                                                'Compras' if 'COMPR' in x.upper() else
                                                                                'Gente e Gestão' if 'GENT' in x.upper() else
                                                                                'Gente e Gestão' if 'DP' in x.upper() else
                                                                                'Estoque' if 'ESTOQ' in x.upper() else
                                                                                'Expedição' if 'EXPED' in x.upper() else
                                                                                'Almoxarifado' if 'ALMOX' in x.upper() else
                                                                                'Financeiro' if 'FINAN' in x.upper() else
                                                                                'Servicos Gerais' if 'GERAIS' in x.upper() else
                                                                                'E-commerce' if 'COMMERCE' in x.upper() else
                                                                                'Inteligencia Comercial' if 'COMERCIAL' in x.upper() else
                                                                                'Cobranca' if 'COBRAN' in x.upper() else
                                                                                'Marketing' if 'MARKET' in x.upper() else
                                                                                'Dados' if 'DADOS' in x.upper() else
                                                                                'Administrativo' if 'ADM' in x.upper() else
                                                                                'Supervisor' if 'CHEFE' in x.upper() else
                                                                                'Tecnologia' if 'TECNO' in x.upper() else
                                                                                'Projetos' if 'PROJE' in x.upper() else
                                                                                'Expansão' if 'EXPAN' in x.upper() else
                                                                                'Motorista' if 'MOTOR' in x.upper() else '')

df_funcionários['Nome completo'] = df_funcionários['Name'] + ' ' + df_funcionários['LastName']

for i in df_funcionários.index:
    nome = []
    for x in df_funcionários.loc[i, 'Nome completo'].split():
        nome_completo = ' '.join(nome)
        df_funcionários.loc[i, 'Nome completo'] = nome_completo

cod_fun = list(df_funcionários[df_funcionários['Enabled'] == 1]['Identifier'])

In [ ]:
contador = 1

df_permis = pd.DataFrame(GeoVictoria.GetPermissions(person=','.join(cod_fun)))
df_horas = pd.DataFrame(GeoVictoria.GetAttendance(person=','.join(cod_fun[:48])))

while contador < int(len(cod_fun) / 48) + 1:
    df_horas = df_horas.append(pd.DataFrame(GeoVictoria.GetAttendance(person=','.join(cod_fun[(contador * 48) + 1:(contador * 48) + 48]))))
    contador = contador + 1

df_horas = df_horas.reset_index(drop=True)

In [ ]:
df_ponto = pd.DataFrame(columns=['Cod. Funcionario',
                                 'Nome completo',
                                 'Turno',
                                 'Permissao',
                                 'Data',
                                 'H. Ent1',
                                 'H. Sai1',
                                 'Intervalo',
                                 'H. Ent2',
                                 'H. Sai2'])

df_ponto['Cod. Funcionario'] = df_horas['identifier']
df_ponto['Data'] = df_horas['begins']
df_ponto['H. Ent1'] = df_horas['first_punch']
df_ponto['H. Sai1'] = df_horas['second_punch']
df_ponto['H. Ent2'] = df_horas['third_punch']
df_ponto['H. Sai2'] = df_horas['fourth_punch']

df_ponto['Data'] = df_ponto['Data'].apply(lambda x: dt.date(int(x[:4]), int(x[4:6]), int(x[6:8])) if len(x) > 10 else None)

df_ponto['H. Ent1'] = df_ponto['H. Ent1'].apply(lambda x: f'{x[8:10]}:{x[10:12]}' if len(x) > 10 else '--:--')
df_ponto['H. Sai1'] = df_ponto['H. Sai1'].apply(lambda x: f'{x[8:10]}:{x[10:12]}' if len(x) > 10 else '--:--')
df_ponto['H. Ent2'] = df_ponto['H. Ent2'].apply(lambda x: f'{x[8:10]}:{x[10:12]}' if len(x) > 10 else '--:--')
df_ponto['H. Sai2'] = df_ponto['H. Sai2'].apply(lambda x: f'{x[8:10]}:{x[10:12]}' if len(x) > 10 else '--:--')

df_ponto['Intervalo'] = df_ponto['H. Ent2'].apply(lambda x: ((int(x[:2]) * 60) + (int(x[3:5]))) if '--:--' != x else 0) - df_ponto['H. Sai1'].apply(lambda x: ((int(x[:2]) * 60) + (int(x[3:5]))) if '--:--' != x else 0)
df_ponto['Intervalo'] = df_ponto['Intervalo'].apply(lambda x: FolhaPonto.minuto_hora(x)).replace('00:00', '--:--')

for i in df_ponto.index:
    t_trabralhado = FolhaPonto.minuto_hora((FolhaPonto.hora_minuto(df_ponto.loc[i, 'H. Sai1']) - FolhaPonto.hora_minuto(df_ponto.loc[i, 'H. Ent1'])) + (FolhaPonto.hora_minuto(df_ponto.loc[i, 'H. Sai2']) - FolhaPonto.hora_minuto(df_ponto.loc[i, 'H. Ent2']))).replace('00:00', '--:--')

    if (df_ponto.loc[i, 'H. Ent1'] != '--:--' and df_ponto.loc[i, 'H. Sai1'] != '--:--') and df_ponto.loc[i, 'H. Ent2'] == '--:--':
        df_ponto.loc[i, 'Situação'] = 'Esqueceu'
    elif (df_ponto.loc[i, 'H. Ent2'] != '--:--' and df_ponto.loc[i, 'H. Sai2'] != '--:--') and df_ponto.loc[i, 'H. Sai1'] == '--:--':
        df_ponto.loc[i, 'Situação'] = 'Esqueceu'
    elif df_ponto.loc[i, 'H. Ent1'] != '--:--' and df_ponto.loc[i, 'H. Sai1'] != '--:--':
        df_ponto.loc[i, 'Situação'] = 'Almoçou'
    elif df_ponto.loc[i, 'H. Ent1'] != '--:--' and df_ponto.loc[i, 'H. Sai2'] != '--:--':
        df_ponto.loc[i, 'Situação'] = 'Saiu direto'
    else:
        df_ponto.loc[i, 'Situação'] = '--:--'
        
    if df_ponto.loc[i, 'Situação'] == 'Esqueceu':
        df_ponto.loc[i, 'Trabalhado'] = '--:--'
    else:
        df_ponto.loc[i, 'Trabalhado'] = t_trabralhado



    weekend_day = df_ponto.loc[i, 'Data'].strftime('%w')
    try:
        df_ponto.loc[i, 'Nome completo'] = df_funcionários[df_funcionários['Identifier'] == df_ponto.loc[i, 'Cod. Funcionario']]['Name'].values[0] + ' ' + df_funcionários[df_funcionários['Identifier'] == df_ponto.loc[i, 'Cod. Funcionario']]['LastName'].values[0]
        
        if (weekend_day != '0') & (weekend_day != '6'):
            df_ponto.loc[i, 'Turno'] = df_funcionários[df_funcionários['Identifier'] == df_ponto.loc[i, 'Cod. Funcionario']]['weeklyHoursCode'].values[0]
        else:
            df_ponto.loc[i, 'Turno'] = 'Nenhum'
            
        for (id, permis) in lista_tp_permis:
            if id == df_permis[(df_permis['Identifier'] == df_ponto.loc[i, 'Cod. Funcionario']) & (df_permis['PERMISSION_START'] == df_ponto.loc[i, 'Data'].strftime('%Y%m%d000000'))]['ID_PERMISSION_TYPE'].values[0]:
                if permis == None:
                    df_ponto.loc[i, 'Permissao'] = 'Nenhum'
                else:
                    df_ponto.loc[i, 'Permissao'] = permis

    except:
        df_ponto.loc[i, 'Permissao'] = 'Nenhum'
        continue

df_ponto = pd.merge(df_data, df_ponto, how='outer', on=['Data'])
df_ponto = df_ponto[df_ponto['Data'] < dt.datetime.now().date()]
# df_ponto.iloc[:, 5:] = df_ponto.iloc[:, 5:].fillna('--:--')

In [ ]:
for i in df_ponto.index:
    if (df_ponto.loc[i, 'Permissao'] == 'Nenhum') & (df_ponto.loc[i, 'Turno'] == '44 horas semanais'):
        cumprimento = '08:48'
    elif (df_ponto.loc[i, 'Permissao'] == 'Nenhum') & (df_ponto.loc[i, 'Turno'] == '35 horas semanais'):
        cumprimento = '07:00'
    elif (df_ponto.loc[i, 'Permissao'] == 'Nenhum') & (df_ponto.loc[i, 'Turno'] == '30 horas semanais '):
        cumprimento = '06:00'
    else:
        cumprimento = '--:--'

    df_ponto.loc[i, 'H. Extra / Faltante'] = FolhaPonto.minuto_hora(FolhaPonto.hora_minuto(df_ponto.loc[i, 'Trabalhado']) - FolhaPonto.hora_minuto(cumprimento)).replace('00:00','--:--')
    cumprimento = '--:--'

df_ponto[(df_ponto['Cod. Funcionario'] == '004064')]# | (df_ponto['Cod. Funcionario'] == '004308')]

In [ ]:
FolhaPonto.calculo(df_ponto[(df_ponto['Cod. Funcionario'] == '004064')]['H. Extra / Faltante'])

In [ ]:
df_funcionários.loc[df_funcionários[df_funcionários['Identifier'] == '003791'].index[0]]